In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import math
from sklearn.model_selection import train_test_split,cross_val_score,KFold,StratifiedKFold
import torchtuples as tt


In [2]:
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder, encode_categorical

from sklearn import set_config
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
datas = pd.read_csv('Simulated_Non_Linear.csv')
datas

,x.1,x.2,x.3,x.4,x.5,time,status
0,0.241721,-0.938963,0.995076,0.873026,-0.619099,0.008877,1
1,0.046930,-2.719881,-0.484127,-0.667630,0.196476,0.003972,1
2,1.235963,-1.879290,0.076798,0.268316,0.351406,0.071261,0
3,0.293341,0.003349,0.942017,0.534047,0.595269,0.012935,1
4,-0.696112,-0.679148,0.271709,0.108177,1.093271,0.310819,1
...,...,...,...,...,...,...,...
495,0.304349,-0.144605,-0.051774,0.024919,-2.218288,0.223093,1
496,-1.079006,-0.199503,-2.160602,-1.945771,-0.394191,0.092461,0
497,0.962852,-1.074593,0.784579,-1.021117,-0.901006,0.222758,0
498,0.958008,-1.034877,0.811625,0.912770,1.298918,0.038232,1


In [4]:
data_y = datas.loc[:, ['time', 'status']].copy()
data_y = data_y.rename(columns={'status': 'cens'})
data_x = datas.drop(['time', 'status'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=1)
list_of_tuples = list(y_train.itertuples(index=False, name=None))
swapped_list = [(t[1], t[0]) for t in list_of_tuples]
y_train = np.array(swapped_list, dtype=[('cens', bool), ('time', float)])
list_of_tuples = list(y_test.itertuples(index=False, name=None))
swapped_list = [(t[1], t[0]) for t in list_of_tuples]
y_test = np.array(swapped_list, dtype=[('cens', bool), ('time', float)])

In [5]:
for df in [X_train, X_test]:
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].min()) / (df[numeric_columns].max() - df[numeric_columns].min())

In [6]:
X_test

,x.1,x.2,x.3,x.4,x.5
304,0.604340,0.140081,0.376403,0.000000,0.565903
340,0.477010,0.780419,0.378433,0.670849,0.569618
47,0.643260,0.716386,0.142990,0.349455,0.611125
67,0.662239,0.644058,0.433874,0.476531,0.523679
479,0.055268,0.295988,0.990846,0.475181,0.480751
...,...,...,...,...,...
331,0.207971,0.664506,0.002574,0.419819,0.914328
197,0.781737,0.042936,0.800449,0.679145,0.625356
95,0.558594,0.298991,0.105625,0.517092,0.467101
122,0.381511,0.541898,0.697024,0.687302,0.575697


# MODELS

In [7]:
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1, alpha_min_ratio=0.01)
cox_lasso.fit(X_train, y_train)
cox_lasso.score(X_test, y_test)

0.5307301698476624

In [8]:
cox_ridge = CoxnetSurvivalAnalysis(l1_ratio=0.00001, alpha_min_ratio=0.01)
cox_ridge.fit(X_train, y_train)
cox_ridge.score(X_test, y_test)

0.5300297671160917

In [9]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

sns.set_style("white")

import torch # For building the networks
import torchtuples as tt # Some useful functions
from torch import nn
import torch.nn.functional as F

from pycox.datasets import support
from pycox.models import DeepHitSingle
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong
from pycox.models.loss import rank_loss_deephit_single

In [10]:
data_y_transform_train = pd.DataFrame.from_records(y_train, columns=['event', 'time'])
data_y_transform_test = pd.DataFrame.from_records(y_test, columns=['event', 'time'])
data_y_transform_train['cens'] = data_y_transform_train['cens'].replace({True: 1, False: 0})
data_y_transform_test['cens'] = data_y_transform_test['cens'].replace({True: 1, False: 0})
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train_values= X_train.values
X_test_values= X_test.values

In [11]:
num_durations = 10

labtrans = DeepHitSingle.label_transform(num_durations)
get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_deephit = labtrans.fit_transform(*get_target(data_y_transform_train))
durations_test, events_test = get_target(data_y_transform_test)

In [12]:
in_features = X_train.shape[1]
out_features = labtrans.out_features
num_nodes = [32]
batch_norm = True
dropout = 0.2

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
optimizer = tt.optim.AdamWR(decoupled_weight_decay=0.01, cycle_eta_multiplier=0.8,
                            cycle_multiplier=2)
model_deep_hit = DeepHitSingle(net, optimizer, alpha = 0.2, sigma = 0.1, duration_index=labtrans.cuts)
epochs = 30
batch_size=8
model_deep_hit.fit(X_train_values, y_train_deephit, batch_size, epochs)
surv = model_deep_hit.predict_surv_df(X_test_values)
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td('antolini')

C:\Users\User\anaconda3\lib\site-packages\torchtuples\callbacks.py:607: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1025.)
  p.data = p.data.add(-weight_decay * eta, p.data)


0:	[0s / 0s],		train_loss: 0.2804
1:	[0s / 0s],		train_loss: 0.2646
2:	[0s / 0s],		train_loss: 0.2613
3:	[0s / 1s],		train_loss: 0.2452
4:	[0s / 1s],		train_loss: 0.2280
5:	[0s / 1s],		train_loss: 0.2286
6:	[0s / 1s],		train_loss: 0.2236
7:	[0s / 1s],		train_loss: 0.2137
8:	[0s / 1s],		train_loss: 0.2070
9:	[0s / 1s],		train_loss: 0.1976
10:	[0s / 2s],		train_loss: 0.1810
11:	[0s / 2s],		train_loss: 0.1779
12:	[0s / 2s],		train_loss: 0.1655
13:	[0s / 2s],		train_loss: 0.1714
14:	[0s / 2s],		train_loss: 0.1661
15:	[0s / 2s],		train_loss: 0.1556
16:	[0s / 3s],		train_loss: 0.1498
17:	[0s / 3s],		train_loss: 0.1723
18:	[0s / 3s],		train_loss: 0.1329
19:	[0s / 3s],		train_loss: 0.1407
20:	[0s / 3s],		train_loss: 0.1334
21:	[0s / 3s],		train_loss: 0.1341
22:	[0s / 3s],		train_loss: 0.1232
23:	[0s / 4s],		train_loss: 0.1182
24:	[0s / 4s],		train_loss: 0.1107
25:	[0s / 4s],		train_loss: 0.1204
26:	[0s / 4s],		train_loss: 0.1112
27:	[0s / 4s],		train_loss: 0.1120
28:	[0s / 4s],		train_loss: 0.

0.5410611101383296

In [13]:
from sksurv.ensemble import RandomSurvivalForest
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           n_jobs=-1,
                           random_state=42)
rsf.fit(X_train,y_train)
rsf.score(X_test, y_test)

0.6270355454386272

In [14]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis().fit(X_train, y_train)
estimator.score(X_test,y_test)

0.5289791630187358

In [15]:
from sksurv.linear_model import CoxnetSurvivalAnalysis
estimator = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
estimator.fit(X_train, y_train)
estimator.score(X_test, y_test)

0.5316056732621257

In [16]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
est_cph_tree = GradientBoostingSurvivalAnalysis(
    n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0
)
est_cph_tree.fit(X_train, y_train)
cindex = est_cph_tree.score(X_test, y_test)
cindex

0.5788828576431448

In [17]:
import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [18]:
get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_deepsurv = get_target(data_y_transform_train)
durations_test, events_test = get_target(data_y_transform_test)

In [19]:
in_features = X_train.shape[1]
num_nodes = [64, 64]
out_features = 1
batch_norm = True
dropout = 0.2
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [21]:
from pycox.models import LogisticHazard
# from pycox.models import PMF
# from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv

In [22]:
num_durations = 10

labtrans = LogisticHazard.label_transform(num_durations)
# labtrans = PMF.label_transform(num_durations)
# labtrans = DeepHitSingle.label_transform(num_durations)

get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_loghazard = labtrans.fit_transform(*get_target(data_y_transform_train))
y_test_loghazard = labtrans.transform(*get_target(data_y_transform_test))


# # We don't need to transform the test labels
durations_test, events_test = get_target(data_y_transform_test)

In [23]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train_values= X_train.values
X_test_values= X_test.values

In [24]:
in_features = X_train.shape[1]
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)
batch_size = 32
epochs = 20
log= model.fit(X_train_values, y_train_loghazard, batch_size, epochs)
surv = model.predict_surv_df(X_test_values)
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td('antolini')

0:	[0s / 0s],		train_loss: 1.1487
1:	[0s / 0s],		train_loss: 0.7790
2:	[0s / 0s],		train_loss: 0.4143
3:	[0s / 0s],		train_loss: 0.2415
4:	[0s / 0s],		train_loss: 0.1862
5:	[0s / 0s],		train_loss: 0.1580
6:	[0s / 0s],		train_loss: 0.1466
7:	[0s / 0s],		train_loss: 0.1382
8:	[0s / 0s],		train_loss: 0.1424
9:	[0s / 0s],		train_loss: 0.1297
10:	[0s / 0s],		train_loss: 0.1387
11:	[0s / 0s],		train_loss: 0.1226
12:	[0s / 0s],		train_loss: 0.1196
13:	[0s / 0s],		train_loss: 0.1371
14:	[0s / 0s],		train_loss: 0.1263
15:	[0s / 0s],		train_loss: 0.1173
16:	[0s / 0s],		train_loss: 0.1086
17:	[0s / 0s],		train_loss: 0.1175
18:	[0s / 0s],		train_loss: 0.0946
19:	[0s / 0s],		train_loss: 0.1021


0.4505340570828226

In [25]:
cox= CoxnetSurvivalAnalysis()
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           random_state=42)
cox.fit(X_train_values, y_train)
rsf.fit(X_train_values, y_train)
cox_pred = cox.predict(X_test_values)
rsf_pred = rsf.predict(X_test_values)
ensemble_pred= (cox_pred+rsf_pred)/2
from sksurv.metrics import concordance_index_censored
cindex = concordance_index_censored(y_test["cens"], y_test["time"], ensemble_pred)
print("Concordance Index:", cindex[0])

Concordance Index: 0.6261600420241639


In [26]:
cox= CoxPHSurvivalAnalysis()
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           random_state=42)
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1, alpha_min_ratio=0.01)
cox_lasso.fit(X_train, y_train)
cox.fit(X_train_values, y_train)
rsf.fit(X_train_values, y_train)
lasso_predict = cox_lasso.predict(X_test_values)
cox_pred = cox.predict(X_test_values)
rsf_pred = rsf.predict(X_test_values)
ensemble_pred = np.maximum(cox_pred, rsf_pred, lasso_predict)
from sksurv.metrics import concordance_index_censored
cindex = concordance_index_censored(y_test["cens"], y_test["time"], ensemble_pred)
print("Concordance Index:", cindex[0])

Concordance Index: 0.6270355454386272


C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but CoxnetSurvivalAnalysis was fitted with feature names
  warnings.warn(
